In [1]:
import pulp as pl

from gurobipy import GRB
from gurobipy import Model

pl.listSolvers(onlyAvailable=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-11-23
No parameters matching '_test' found


['GLPK_CMD', 'GUROBI', 'GUROBI_CMD', 'PULP_CBC_CMD', 'PULP_CHOCO_CMD']

The following is the original question shown in the quiz, howerver, the primal is infesiable as it is. We need to change the non-positive constraint of $x_2$ to non-negative, in the actual computation.

![image](./quiz.png)

In [2]:
model_gurobi = Model(name="week-2-quiz")

x1 = model_gurobi.addVar(lb=0, vtype=GRB.CONTINUOUS, name="x1")
x2 = model_gurobi.addVar(lb=0, vtype=GRB.CONTINUOUS, name="x2")
x3 = model_gurobi.addVar(lb=-1e100, vtype=GRB.CONTINUOUS, name="x3")

model_gurobi.setObjective(expr=4 * x1 - 2 * x2 + x3, sense=GRB.MAXIMIZE)
model_gurobi.addConstr(2 * x1 + x2 <= 10, "constraint_1")
model_gurobi.addConstr(x2 + x3 >= -3, "constraint_2")
model_gurobi.addConstr(x1 + 3 * x2 - 3 * x3 == 14, "constraint_3")

model_gurobi.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 3 rows, 3 columns and 7 nonzeros
Model fingerprint: 0x323a8293
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 1e+01]
Presolve removed 1 rows and 1 columns
Presolve time: 0.08s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7000000e+01   0.000000e+00   0.000000e+00      0s
       0    1.7000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.14 seconds (0.00 work units)
Optimal objective  1.700000000e+01


In [3]:
# shadow prices
model_gurobi.Pi

[2.1666666666666665, 0.0, -0.3333333333333333]

In [4]:
model = pl.LpProblem(name="week-2-quiz", sense=pl.LpMaximize)

# Decision variables
x1 = pl.LpVariable(name="x1", lowBound=0)
x2 = pl.LpVariable(name="x2", lowBound=0)
x3 = pl.LpVariable(name="x3")

# Objective function
model += pl.lpSum([4 * x1, -2 * x2, x3])

# Constraints
model += (2 * x1 + x2 <= 10, "constraint_1")
model += (x2 + x3 >= -3, "constraint_2")
model += (x1 + 3 * x2 - 3 * x3 == 14, "constraint_3")

_ = model.solve(solver=pl.PULP_CBC_CMD())

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/changyaochen/.local/share/virtualenvs/Coursera-operation-research-3-RVohkWLL/lib/python3.7/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ff/lzxbccs579ldzr_264f20rtr0000gn/T/16bb26ca2d8248aab335f3ca5de84d5e-pulp.mps max branch printingOptions all solution /var/folders/ff/lzxbccs579ldzr_264f20rtr0000gn/T/16bb26ca2d8248aab335f3ca5de84d5e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 19 RHS
At line 23 BOUNDS
At line 25 ENDATA
Problem MODEL has 3 rows, 3 columns and 7 elements
Coin0008I MODEL read with 0 errors
Presolve 0 (-3) rows, 0 (-3) columns and 0 (-7) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 17
After Postsolve, objective 17, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 17 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal

In [5]:
for v in model.variables():
    print(f"The value of {v} is: {v.value()}")

for name, c in list(model.constraints.items()):
    print(f"For constraint: {c}, its shadow price is {c.pi}, with slack of {c.slack}")

The value of x1 is: 5.0
The value of x2 is: 0.0
The value of x3 is: -3.0
For constraint: 2*x1 + x2 <= 10, its shadow price is 2.1666667, with slack of -0.0
For constraint: x2 + x3 >= -3, its shadow price is -0.0, with slack of -0.0
For constraint: x1 + 3*x2 - 3*x3 = 14, its shadow price is -0.33333333, with slack of -0.0
